In [25]:
import numpy as np
import geopandas as gpd

def calc_iou(gdf_groundtruth, gdf_predict):
    """Calculates intersection over union (iou) score.

    Args:
      gdf_groundtruth: Groundtruth GeoDataFrame of polygons.
      gdf_predict: Predicted GeoDataFrame of polygons.

    Returns:
      Intersection Over Union (IOU) Score.

    """
    
    intersect = gdf_groundtruth.dissolve().intersection(gdf_predict.dissolve()).area
    union = gdf_groundtruth.dissolve().union(gdf_predict.dissolve()).area
    iou = intersect / union
    
    return iou[0]

def calc_datasets(groundtruth_file, predict_file):
    """Reads geojson files, and calculates IOU.

    Args:
      groundtruth_file: Geojson file for groundtruth.
      predict_file: Geojson file for predictions.

    Returns:
      Intersection over union (IOU) score with punishment.
    """
    
    gdf_groundtruth = gpd.read_file(groundtruth_file)
    gdf_predict = gpd.read_file(predict_file)
    
    # Validate the CRS
    assert gdf_predict.crs==3857, (
        f'All geometries must be in EPSG:3857 Coordinate Reference System.')
    assert gdf_groundtruth.crs==3857, (
        f'All geometries must be in EPSG:3857 Coordinate Reference System.')
    
    # Validate the geometry column
    assert "geometry" in gdf_predict.columns, (
        f'Missing geometry column.')
    assert "geometry" in gdf_groundtruth.columns, (
        f'Missing geometry column.')
    
    iou = calc_iou(gdf_groundtruth, gdf_predict)
    # Punish if more polygon provided 
    if len(gdf_groundtruth)<len(gdf_predict):
        iou = iou * (len(gdf_groundtruth)/len(gdf_predict))
    
    return iou
    
def evaluate(groundtruth_files, predict_files):
    """Calculates datasets for multiple files.

    Args:
      groundtruth_files: List of groundtruth files.
      predict_files: List of prediction files. One file for each groundtruth.

    Returns:
      Intersection Over Union (IOU) Score.
    """
    
    # Check the number of files provided
    assert len(groundtruth_files) == len(predict_files), (
        f'Number of files are not matching.')
    
    results = []
    
    for i in range(len(groundtruth_files)):
        iou = calc_datasets(groundtruth_files[i], predict_files[i])
        results.append(iou)
    
    return np.mean(results)

In [ ]:
#10=>15=>28=>35=>
#with the smalles dataset of 8 mb we get 15 seconds for dsm generation 10 seconds for dtm genreation while ndhm generation
#and most of our algorithms give us instentenous outputs while when we add tri we get 8 secons of calcualtion time

calc_datasets("USGS_LPC_IL_HicksDome_FluorsparDistrict_2019_D19_2339_5650_gt_buildings.geojson","building2.geojson")

In [26]:
dataset=["USGS_LPC_IL_8County_2020_A20_1035_8430_gt_buildings.geojson","USGS_LPC_IL_HicksDome_FluorsparDistrict_2019_D19_2339_5650_gt_buildings.geojson","USGS_LPC_MI_16Co_Lapeer_2015_460530_LAS_2018_gt_buildings.geojson","USGS_LPC_MI_16Co_Shiawassee_2015_235540_LAS_2018_gt_buildings.geojson","USGS_LPC_MI_WayneCo_2017_382227_LAS_2018_gt_buildings.geojson"]
output_json=["building1.geojson","building2.geojson","building3.geojson","building4.geojson","building5.geojson"]
thresh_json=["1_img_thresh.geojson","2_img_thresh.geojson","3_img_thresh.geojson","4_img_thresh.geojson","5_img_thresh.geojson"]
open_json=["1_img_open.geojson","2_img_open.geojson","3_img_open.geojson","4_img_open.geojson","5_img_open.geojson"]
cont_json=["1_building_masknotri.geojson","2_building_masknotri.geojson","3_building_masknotri.geojson","4_building_masknotri.geojson","5_building_masknotri.geojson",]
tri_json=["1_building_mask.geojson","2_building_mask.geojson","3_building_mask.geojson","4_building_mask.geojson","5_building_mask.geojson"]

In [39]:
open1=calc_datasets(dataset[0],open_json[0])
cont1=calc_datasets(dataset[0],cont_json[0])
tri1=calc_datasets(dataset[0],tri_json[0])
out1=calc_datasets(dataset[0],output_json[0])

In [40]:
open2=calc_datasets(dataset[1],open_json[1])
cont2=calc_datasets(dataset[1],cont_json[1])
tri2=calc_datasets(dataset[1],tri_json[1])
out2=calc_datasets(dataset[1],output_json[1])

In [41]:
open3=calc_datasets(dataset[2],open_json[2])
cont3=calc_datasets(dataset[2],cont_json[2])
tri3=calc_datasets(dataset[2],tri_json[2])
out3=calc_datasets(dataset[2],output_json[2])

In [42]:
open4=calc_datasets(dataset[3],open_json[3])
cont4=calc_datasets(dataset[3],cont_json[3])
tri4=calc_datasets(dataset[3],tri_json[3])
out4=calc_datasets(dataset[3],output_json[3])

In [43]:
open5=calc_datasets(dataset[4],open_json[4])
cont5=calc_datasets(dataset[4],cont_json[4])
tri5=calc_datasets(dataset[4],tri_json[4])
out5=calc_datasets(dataset[4],output_json[4])

In [44]:
import csv
import numpy as np


data = [
    {
        'open': open1,
        'cont': cont1,
        'tri': tri1,
        'closed': out1,
    },
    {
        'open': open2,
        'cont': cont2,
        'tri': tri2,
        'closed': out2,
    },
    {
        'open': open3,
        'cont': cont3,
        'tri': tri3,
        'closed': out3,
    },
    {
        'open': open4,
        'cont': cont4,
        'tri': tri4,
        'closed': out4,
    },
    {
        'open': open5,
        'cont': cont5,
        'tri': tri5,
        'closed': out5,
    },
]

with open('output.csv', 'w', newline='') as csvfile:
    fieldnames = ['open', 'cont', 'tri', 'closed']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in data:
        writer.writerow(row)
